In [2]:
import pandas as pd
import numpy as np

In [5]:
iris=pd.read_csv('Iris.csv')

In [6]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [10]:
iris['Species'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [13]:
iris_setosa=iris[iris['Species']=='Iris-setosa']
iris_versicolor=iris[iris['Species']=='Iris-versicolor']
iris_virginica=iris[iris['Species']=='Iris-virginica']

In [42]:
iris_versicolor.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
50,51,7.0,3.2,4.7,1.4,Iris-versicolor
51,52,6.4,3.2,4.5,1.5,Iris-versicolor
52,53,6.9,3.1,4.9,1.5,Iris-versicolor
53,54,5.5,2.3,4.0,1.3,Iris-versicolor
54,55,6.5,2.8,4.6,1.5,Iris-versicolor


Me calculo los $ \#C_k /N$:

In [18]:
N_setosa=len(iris_setosa)/len(iris)
N_versicolor=len(iris_versicolor)/len(iris)
N_virginica=len(iris_virginica)/len(iris)

N_setosa, N_versicolor, N_virginica

(0.3333333333333333, 0.3333333333333333, 0.3333333333333333)

In [22]:
iris_setosa.describe()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,50.00000,50.00000,50.000000,50.000000,50.00000
mean,25.50000,5.00600,3.418000,1.464000,0.24400
std,14.57738,0.35249,0.381024,0.173511,0.10721
min,1.00000,4.30000,2.300000,1.000000,0.10000
25%,13.25000,4.80000,3.125000,1.400000,0.20000
50%,25.50000,5.00000,3.400000,1.500000,0.20000
75%,37.75000,5.20000,3.675000,1.575000,0.30000
max,50.00000,5.80000,4.400000,1.900000,0.60000


Me defino la función `gauss(x,mu,sigma)` que da
$$
\mathcal{N}(x|\mu,\sigma)=\frac{1}{\sqrt{2\pi\sigma}} e^{-\frac{(x-\mu)^2}{2\sigma^2}}
$$

In [38]:
def gauss(x,mu,sigma):
    return np.exp(-(x-mu)**2/(2*sigma**2))/(np.sqrt(2*np.pi*sigma))

Mi nueva instancia $x$ será simplemente la primera fila del dataset

In [26]:
x = iris.iloc[0]

In [27]:
x

Id                         1
SepalLengthCm            5.1
SepalWidthCm             3.5
PetalLengthCm            1.4
PetalWidthCm             0.2
Species          Iris-setosa
Name: 0, dtype: object

In [31]:
iris_setosa['SepalLengthCm'].var()

0.12424897959183666

Me calculo lo que quiero maximizar:
$$
    \# C_k / N_k \cdot \prod_i \mathcal{N}(x_i|\mu_k^i,\sigma_k^i)
    $$

In [33]:
iris.columns[1:5]

Index(['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'], dtype='object')

In [39]:
P_setosa=N_setosa
for i in iris.columns[1:5]:
    P_setosa=P_setosa*gauss(x[i],iris_setosa[i].mean(),iris_setosa[i].var())
    
P_setosa

0.00014852476867097342

In [40]:
P_virginica=N_virginica
for i in iris.columns[1:5]:
    P_virginica=P_virginica*gauss(x[i],iris_virginica[i].mean(),iris_virginica[i].var())
    
P_virginica

2.2129917787430672e-177

$$ 2.21 \cdot 10 ^{-177}$$

In [41]:
P_versicolor=N_versicolor
for i in iris.columns[1:5]:
    P_versicolor=P_versicolor*gauss(x[i],iris_versicolor[i].mean(),iris_versicolor[i].var())
    
P_versicolor

1.6634597254045788e-231

Predigo que es de tipo Iris-setosa

### Ejercicio
Automatizar todo esto.

Hagámoslo ahora automáticamente con sklearn

In [45]:
X=iris[iris.columns[1:5]]
X.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [47]:
y=iris['Species']

Separamos en train y test

In [67]:
from sklearn.model_selection import train_test_split

In [68]:
train_X,val_X,train_y,val_y = train_test_split(X,y,random_state=1)

In [69]:
from sklearn.naive_bayes import GaussianNB

In [70]:
# Definimos
gnb = GaussianNB()
# Ajsutamos
gnb.fit(train_X,train_y)
# Predecimos
y_pred = gnb.predict(val_X)

In [71]:
print('Número de puntos mal asignados de un total de %d puntos: %d' % (len(val_X),(val_y!=y_pred).sum()))

Número de puntos mal asignados de un total de 38 puntos: 1


In [72]:
val_y

14         Iris-setosa
98     Iris-versicolor
75     Iris-versicolor
16         Iris-setosa
131     Iris-virginica
56     Iris-versicolor
141     Iris-virginica
44         Iris-setosa
29         Iris-setosa
120     Iris-virginica
94     Iris-versicolor
5          Iris-setosa
102     Iris-virginica
51     Iris-versicolor
78     Iris-versicolor
42         Iris-setosa
92     Iris-versicolor
66     Iris-versicolor
31         Iris-setosa
35         Iris-setosa
90     Iris-versicolor
84     Iris-versicolor
77     Iris-versicolor
40         Iris-setosa
125     Iris-virginica
99     Iris-versicolor
33         Iris-setosa
19         Iris-setosa
73     Iris-versicolor
146     Iris-virginica
91     Iris-versicolor
135     Iris-virginica
69     Iris-versicolor
128     Iris-virginica
114     Iris-virginica
48         Iris-setosa
53     Iris-versicolor
28         Iris-setosa
Name: Species, dtype: object

In [73]:
y_pred

array(['Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa',
       'Iris-virginica', 'Iris-versicolor', 'Iris-virginica',
       'Iris-setosa', 'Iris-setosa', 'Iris-virginica', 'Iris-versicolor',
       'Iris-setosa', 'Iris-virginica', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-setosa', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-virginica', 'Iris-setosa',
       'Iris-virginica', 'Iris-versicolor', 'Iris-setosa', 'Iris-setosa',
       'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor',
       'Iris-virginica', 'Iris-versicolor', 'Iris-virginica',
       'Iris-virginica', 'Iris-setosa', 'Iris-versicolor', 'Iris-setosa'],
      dtype='<U15')

In [74]:
(val_y != y_pred)

14     False
98     False
75     False
16     False
131    False
56     False
141    False
44     False
29     False
120    False
94     False
5      False
102    False
51     False
78     False
42     False
92     False
66     False
31     False
35     False
90     False
84     False
77      True
40     False
125    False
99     False
33     False
19     False
73     False
146    False
91     False
135    False
69     False
128    False
114    False
48     False
53     False
28     False
Name: Species, dtype: bool